## Setup

In [1]:
%%capture
try:
    import google.colab # type: ignore
    from google.colab import output
    %pip install sae-lens==1.3.0 transformer-lens==1.17.0 circuitsvis==1.43.2
except:
    from IPython import get_ipython # type: ignore
    ipython = get_ipython(); assert ipython is not None
    ipython.run_line_magic("load_ext", "autoreload")
    ipython.run_line_magic("autoreload", "2")

In [2]:
import torch
import os

from sae_lens.training.config import LanguageModelSAERunnerConfig
from sae_lens.training.lm_runner import language_model_sae_runner

if torch.cuda.is_available():
    device = "cuda"
elif torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("Using device:", device)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

Using device: cuda


# Model Selection and Evaluation


In [3]:
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained(
    "gpt2-xl"
)  # This will wrap huggingface models and has lots of nice utilities.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/689 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-xl into HookedTransformer


### Getting a vibe for a model using `model.generate`

Let's start by generating some stories using the model.

In [4]:
# # here we use generate to get 10 completeions with temperature 1. Feel free to play with the prompt to make it more interesting.
# for i in range(2):
#     display(
#         model.generate(
#             # "I think you're",
#             "You messed up because you're",
#             stop_at_eos=False,  # avoids a bug on MPS
#             temperature=1,
#             verbose=False,
#             max_new_tokens=50,
#         )
#     )

"You messed up because you're dishonest; honest people will accept that I know what I'm doing. If you don't, you can try again next week, next month, or next year. Or do us all a favor and fuck yourself\n\nI can't tell you to"

"You messed up because you're not good enough. At some time you have to unlearn the ways you've learned. At some point you have to learn new ways.\n\n\nThe treatment of Walter White's epiphany was truly brilliant. It was unusual, yes. He certainly"

One thing we notice is that the model seems to be able to repeat [X] consistently. To better understand the models ability to remember [X], let's extract a prompt where the next character is determined and use the "test_prompt" utility from TransformerLens to check the ranking of the token for [X].

### Spot checking model abilities with `transformer_lens.utils.test_prompt`

In [5]:
# from transformer_lens.utils import test_prompt

# # Test the model with a prompt
# test_prompt(
#     "I think you're",
#     " angry",
#     model,
#     prepend_space_to_answer=False,
# )

Tokenized prompt: ['<|endoftext|>', 'I', ' think', ' you', "'re"]
Tokenized answer: [' angry']


Performance on answer token:
Rank: 248      Logit:  9.03 Prob:  0.03% Token: | angry|

Top 0th token. Logit: 15.02 Prob: 10.41% Token: | right|
Top 1th token. Logit: 14.38 Prob:  5.48% Token: | missing|
Top 2th token. Logit: 14.27 Prob:  4.89% Token: | being|
Top 3th token. Logit: 13.88 Prob:  3.32% Token: | going|
Top 4th token. Logit: 13.76 Prob:  2.94% Token: | a|
Top 5th token. Logit: 13.47 Prob:  2.20% Token: | making|
Top 6th token. Logit: 13.46 Prob:  2.18% Token: | getting|
Top 7th token. Logit: 13.42 Prob:  2.09% Token: | talking|
Top 8th token. Logit: 13.41 Prob:  2.07% Token: | looking|
Top 9th token. Logit: 13.39 Prob:  2.03% Token: | on|


Ranks of the answer tokens: [(' angry', 248)]

In the output above, we see that the model assigns ~ % probability to [X] being the next token.

### Exploring Model Capabilities with Log Probs

Look at token log probs for ALL tokens in a prompt. Hover to get the top5 tokens by log probability. Darker tokens are tokens where the model assigned a higher probability to the actual next token.

Given prompt "A B C D", this predicts the rank of predicting "C" given "A B". The actual prompt has "A B C", but if only "A B" was given, how "much" does the model expect C? [improve this explanation]

In [6]:
# import circuitsvis as cv  # optional dep, install with pip install circuitsvis

# # Let's make a longer prompt and see the log probabilities of the tokens
# example_prompt = """Hi, how are you doing this? I'm really enjoying your posts"""
# logits, cache = model.run_with_cache(example_prompt)
# cv.logits.token_log_probs(
#     model.to_tokens(example_prompt),
#     model(example_prompt)[0].log_softmax(dim=-1),
#     model.to_string,
# )
# # hover on the output to see the result.

Let's combine `model.generate` and the token log probs visualization to see the log probs on text generated by the model. Note that we can play with the temperature and this should sample less likely trajectories according to the model.

Some things to explore:
- Which tokens does the model assign high probability to? Can you see how the model should know which word comes next?
- What happens if you increase / decrease the temperature?
- Do the rankings of tokens seem sensible to you? What about where the model doesn't assign a high probability to the token which came next?

In [7]:
# example_prompt = model.generate(
#     "You messed up because you're",
#     stop_at_eos=False,  # avoids a bug on MPS
#     temperature=1,
#     verbose=True,
#     max_new_tokens=50,
# )
# logits, cache = model.run_with_cache(example_prompt)
# cv.logits.token_log_probs(
#     model.to_tokens(example_prompt),
#     model(example_prompt)[0].log_softmax(dim=-1),
#     model.to_string,
# )

  0%|          | 0/50 [00:00<?, ?it/s]

# Training an SAE

Now we're ready to train out SAE. We'll make a runner config, instantiate the runner and the rest is taken care of for us!

During training, you use weights and biases to check key metrics which indicate how well we are able to optimize the variables we care about.

To get a better sense of which variables to look at, you can read my (Joseph's) post [here](https://www.lesswrong.com/posts/f9EgfLSurAiqRJySD/open-source-sparse-autoencoders-for-all-residual-stream) and especially look at my weights and biases report [here](https://links-cdn.wandb.ai/wandb-public-images/links/jbloom/uue9i416.html).

A few tips:
- Feel free to reorganize your wandb dashboard to put L0, CE_Loss_score, explained variance and other key metrics in one section at the top.
- Make a [run comparer](https://docs.wandb.ai/guides/app/features/panels/run-comparer) when tuning hyperparameters.
- You can download the resulting sparse autoencoder / sparsity estimate from wandb and upload them to huggingface if you want to share your SAE with other.
    - cfg.json (training config)
    - sae_weight.safetensors (model weights)
    - sparsity.safetensors (sparsity estimate)

## MLP Out

I've tuned the hyperparameters below for a decent SAE which achieves 86% CE Loss recovered and an L0 of ~85, and runs in about 2 hours on an M3 Max. You can get an SAE that looks better faster if you only consider L0 and CE loss but it will likely have more dense features and more dead features. Here's a link to my output with two runs with two different L1's: https://wandb.ai/jbloom/sae_lens_tutorial .

Paste wandb API key below

In [4]:
# total_training_steps = 30_000  # probably we should do more
total_training_steps = 1000  # probably we should do more
batch_size = 4096
# batch_size = 4
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name="gpt2-xl",  # our model (more options here: https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
    # hook_point="blocks.20.hook_mlp_out",  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    # hook_point_layer=20,  # Only one layer in the model.
    hook_point="blocks.5.hook_mlp_out",  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    hook_point_layer=5,  # Only one layer in the model.
    # d_in=1024,  # the width of the mlp output.
    d_in=1600,  # the width of the mlp output.
    # dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    dataset_path="stas/openwebtext-10k",
    is_dataset_tokenized=True,
    # streaming=True,  # we could pre-download the token dataset if it was small.

    # SAE Parameters
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor=16,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="zeros",  # The geometric median can be used to initialize the decoder weights.
    apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
    normalize_sae_decoder=False,
    # scale_sparsity_penalty_by_decoder_norm=True,
    # decoder_heuristic_init=True,
    # init_encoder_as_decoder_transpose=True,
    # normalize_activations=False,

    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    l1_coefficient=5,  # will control how sparse the feature activations are
    # l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    # train_batch_size_tokens=batch_size,
    context_size=256,  # will control the lenght of the prompts we feed to the model. Larger is better but slower. so for the tutorial we'll use a short one.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    # store_batch_size_prompts=16,

    # Resampling protocol
    use_ghost_grads=False,  # we don't use ghost grads anymore.
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.

    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    # log_to_wandb=False,
    wandb_project="sae_lens_exploraTest_L5",
    # wandb_project="sae_lens_tutorial",
    wandb_log_frequency=30,
    # eval_every_n_wandb_logs=20,

    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype=torch.float32,
)

# look at the next cell to see some instruction for what to do while this is running.
sparse_autoencoder_dictionary = language_model_sae_runner(cfg)

Run name: 25600-L1-5-LR-5e-05-Tokens-4.096e+06
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.002048
Total training steps: 1000
Total wandb updates: 33
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 1048.576
We will reset the sparsity calculation 1 times.
Number tokens in sparsity calculation window: 4.10e+06
Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda
Run name: 25600-L1-5-LR-5e-05-Tokens-4.096e+06
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.002048
Total training steps: 1000
Total wandb updates: 33
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 1048.576
We will reset the sparsity calculation 1 times.
Number tokens in sparsity calculation window: 4.10e+06
Run name: 25600-L1-5-LR-5e-05-Tokens-4.096e+06
n_tokens_per_buffer (millions): 0.524288
Lower b

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


1000| MSE Loss 0.073 | L1 19.948: 100%|██████████| 4096000/4096000 [08:10<00:00, 28219.64it/s]

details/current_learning_rate,███████████████████████████▇▅▄▃▂▁
details/n_training_tokens,▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇███
losses/ghost_grad_loss,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/l1_loss,█▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
losses/mse_loss,█▅▄▃▂▂▂▁▂▁▂▂▁▁▁▁▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▁▁
losses/overall_loss,█▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
metrics/CE_loss_score,▁▄█
metrics/ce_loss_with_ablation,▅▁█
metrics/ce_loss_with_sae,▅▁█
metrics/ce_loss_without_sae,▅▁█
metrics/explained_variance,▁▅▆▇▇████████████████████████████


In [4]:
# total_training_steps = 30_000  # probably we should do more
total_training_steps = 1000  # probably we should do more
batch_size = 4096
# batch_size = 4
total_training_tokens = total_training_steps * batch_size

lr_warm_up_steps = 0
lr_decay_steps = total_training_steps // 5  # 20% of training
l1_warm_up_steps = total_training_steps // 20  # 5% of training

cfg_2 = LanguageModelSAERunnerConfig(
    # Data Generating Function (Model + Training Distibuion)
    model_name="gpt2-xl",  # our model (more options here: https://neelnanda-io.github.io/TransformerLens/generated/model_properties_table.html)
    # hook_point="blocks.20.hook_mlp_out",  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    # hook_point_layer=20,  # Only one layer in the model.
    hook_point="blocks.5.hook_mlp_out",  # A valid hook point (see more details here: https://neelnanda-io.github.io/TransformerLens/generated/demos/Main_Demo.html#Hook-Points)
    hook_point_layer=5,  # Only one layer in the model.
    # d_in=1024,  # the width of the mlp output.
    d_in=6400,  # the width of the mlp output.
    # dataset_path="apollo-research/roneneldan-TinyStories-tokenizer-gpt2",  # this is a tokenized language dataset on Huggingface for the Tiny Stories corpus.
    dataset_path="stas/openwebtext-10k",
    is_dataset_tokenized=True,
    # streaming=True,  # we could pre-download the token dataset if it was small.

    # SAE Parameters
    mse_loss_normalization=None,  # We won't normalize the mse loss,
    expansion_factor=16,  # the width of the SAE. Larger will result in better stats but slower training.
    b_dec_init_method="zeros",  # The geometric median can be used to initialize the decoder weights.
    apply_b_dec_to_input=False,  # We won't apply the decoder weights to the input.
    normalize_sae_decoder=False,
    # scale_sparsity_penalty_by_decoder_norm=True,
    # decoder_heuristic_init=True,
    # init_encoder_as_decoder_transpose=True,
    # normalize_activations=False,

    # Training Parameters
    lr=5e-5,  # lower the better, we'll go fairly high to speed up the tutorial.
    adam_beta1=0.9,  # adam params (default, but once upon a time we experimented with these.)
    adam_beta2=0.999,
    lr_scheduler_name="constant",  # constant learning rate with warmup. Could be better schedules out there.
    lr_warm_up_steps=lr_warm_up_steps,  # this can help avoid too many dead features initially.
    lr_decay_steps=lr_decay_steps,  # this will help us avoid overfitting.
    l1_coefficient=5,  # will control how sparse the feature activations are
    # l1_warm_up_steps=l1_warm_up_steps,  # this can help avoid too many dead features initially.
    lp_norm=1.0,  # the L1 penalty (and not a Lp for p < 1)
    # train_batch_size_tokens=batch_size,
    context_size=256,  # will control the lenght of the prompts we feed to the model. Larger is better but slower. so for the tutorial we'll use a short one.
    # Activation Store Parameters
    n_batches_in_buffer=64,  # controls how many activations we store / shuffle.
    training_tokens=total_training_tokens,  # 100 million tokens is quite a few, but we want to see good stats. Get a coffee, come back.
    # store_batch_size_prompts=16,

    # Resampling protocol
    use_ghost_grads=False,  # we don't use ghost grads anymore.
    feature_sampling_window=1000,  # this controls our reporting of feature sparsity stats
    dead_feature_window=1000,  # would effect resampling or ghost grads if we were using it.
    dead_feature_threshold=1e-4,  # would effect resampling or ghost grads if we were using it.

    # WANDB
    log_to_wandb=True,  # always use wandb unless you are just testing code.
    # log_to_wandb=False,
    wandb_project="sae_lens_exploraTest_L5_v2",
    # wandb_project="sae_lens_tutorial",
    wandb_log_frequency=30,
    # eval_every_n_wandb_logs=20,

    # Misc
    device=device,
    seed=42,
    n_checkpoints=0,
    checkpoint_path="checkpoints",
    dtype=torch.float32,
)

# look at the next cell to see some instruction for what to do while this is running.
sparse_autoencoder_dictionary_2 = language_model_sae_runner(cfg_2)

Run name: 102400-L1-5-LR-5e-05-Tokens-4.096e+06
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.002048
Total training steps: 1000
Total wandb updates: 33
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 1048.576
We will reset the sparsity calculation 1 times.
Number tokens in sparsity calculation window: 4.10e+06


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loaded pretrained model gpt2-xl into HookedTransformer
Moving model to device:  cuda
Run name: 102400-L1-5-LR-5e-05-Tokens-4.096e+06
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.002048
Total training steps: 1000
Total wandb updates: 33
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 1048.576
We will reset the sparsity calculation 1 times.
Number tokens in sparsity calculation window: 4.10e+06
Run name: 102400-L1-5-LR-5e-05-Tokens-4.096e+06
n_tokens_per_buffer (millions): 0.524288
Lower bound: n_contexts_per_buffer (millions): 0.002048
Total training steps: 1000
Total wandb updates: 33
n_tokens_per_feature_sampling_window (millions): 1048.576
n_tokens_per_dead_feature_window (millions): 1048.576
We will reset the sparsity calculation 1 times.
Number tokens in sparsity calculation window: 4.10e+06


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Training SAE:   0%|          | 0/4096000 [00:00<?, ?it/s]

RuntimeError: The expanded size of the tensor (6400) must match the existing size (1600) at non-singleton dimension 3.  Target sizes: [32, 256, 1, 6400].  Tensor sizes: [32, 256, 1, 1600]

# Interpret SAE


In [5]:
import pandas as pd

# Let's start by getting the top 10 logits for each feature

sparse_autoencoder = next(iter(sparse_autoencoder_dictionary))[1]
projection_onto_unembed = sparse_autoencoder.W_dec @ model.W_U


# get the top 10 logits.
vals, inds = torch.topk(projection_onto_unembed, 10, dim=1)

# get 10 random features
random_indices = torch.randint(0, projection_onto_unembed.shape[0], (10,))

# Show the top 10 logits promoted by those features
top_10_logits_df = pd.DataFrame(
    [model.to_str_tokens(i) for i in inds[random_indices]],
    index=random_indices.tolist(),
).T
top_10_logits_df

,18319,22701,9247,12551,14780,10417,2686,3515,5326,6577
0,izen,Ether,anza,Symbol,voc,Patton,anza,emate,FD,nar
1,Oath,Hau,harmless,lore,Registered,Fallon,INT,oqu,SEA,Reid
2,Pact,Silence,nap,crit,house,ZA,WHERE,Hast,Ley,idal
3,ilee,Frame,alez,cyclop,MV,WN,itia,veto,arte,Marriott
4,Folder,Wire,iP,Hat,Maria,Panel,Mu,eering,Marina,nar
5,Berry,Noble,opener,wig,uses,Somers,Blu,orically,FD,resolution
6,asin,Ashe,lic,anni,stem,icio,ESC,SPONSORED,Ferry,prev
7,Sys,sans,Duck,Nap,Su,Kahn,Camera,mol,FTA,channel
8,Minute,Lug,fav,mil,Mp,VD,eligible,hast,Underworld,nan
9,ember,Fargo,Chick,Grav,holders,Wilde,RES,Whitman,emulation,resolution


In [ ]:
import pandas as pd

# Let's start by getting the top 10 logits for each feature

sparse_autoencoder = next(iter(sparse_autoencoder_dictionary_2))[1]
projection_onto_unembed = sparse_autoencoder.W_dec @ model.W_U


# get the top 10 logits.
vals, inds = torch.topk(projection_onto_unembed, 10, dim=1)

# get 10 random features
random_indices = torch.randint(0, projection_onto_unembed.shape[0], (10,))

# Show the top 10 logits promoted by those features
top_10_logits_df = pd.DataFrame(
    [model.to_str_tokens(i) for i in inds[random_indices]],
    index=random_indices.tolist(),
).T
top_10_logits_df

# Load SAE

In [ ]:
# run = wandb.init(project="sae_lens_exploraTest_L5", id="cguht60b", resume="allow")
# artifact = run.use_artifact('wlg100/sae_lens_exploraTest_L5/model:v0', type='model')
# model_dir = artifact.download()

# # Load your model from the downloaded directory
# import torch
# model = torch.load(model_dir + "/model.pth")

In [ ]:
# from sae_lens import LMSparseAutoencoderSessionloader

# path ="path/to/sparse_autoencoder.pt"
# model, sparse_autoencoder, activations_loader = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
#     path
# )

# Steering Vector decomposition

In [36]:
# do this b/c anger is one token, calm is 2, so this pads anger with 50256
batch_input = ["anger", "calm"]
tokens = model.to_tokens(batch_input)
tokens

tensor([[50256,  2564, 50256],
        [50256,  9948,    76]], device='cuda:0')

In [8]:
import torch as t

In [51]:
layer_name = 'blocks.5.mlp.hook_pre'

In [31]:
model.cfg.d_model

1600

In [14]:
from jaxtyping import Float, Int
from torch import nn, Tensor

In [46]:
def store_h_hook(
    pattern: Float[Tensor, "batch seqlen d_mlp"],
    # hook: HookPoint,
    hook
):
    # Store the result.
    # h_store = pattern  # this won't work b/c replaces entire thing, so won't be stored
    # h_store.append(1) # if h_store = [], this will work
    h_store[:] = pattern  # this works b/c changes values, not replaces entire thing

In [53]:
seqLen = tokens.shape[1]
h_store = t.zeros((1, seqLen, model.cfg.d_mlp), device=model.cfg.device)
# h_store = t.zeros((len(batch_input), seqLen, model.cfg.d_model), device=model.cfg.device)

model.run_with_hooks(
    tokens[0],
    return_type = None,
    fwd_hooks=[
        (layer_name, store_h_hook),
    ]
)

neg_h = t.clone(h_store)

RuntimeError: The expanded size of the tensor (1600) must match the existing size (6400) at non-singleton dimension 2.  Target sizes: [2, 3, 1600].  Tensor sizes: [3, 6400]

In [17]:
seqLen = tokens.shape[1]
h_store = t.zeros((1, seqLen, model.cfg.d_mlp), device=model.cfg.device)

model.run_with_hooks(
    tokens[1],
    return_type = None,
    fwd_hooks=[
        (layer_name, store_h_hook),
    ]
)

pos_h = t.clone(h_store)

In [18]:
steer_vec = neg_h - pos_h
steer_vec.shape

torch.Size([1, 3, 6400])

In [19]:
import einops

In [20]:
# get LLM activs for steering vec
post_reshaped = einops.repeat(steer_vec, "batch seq d_mlp -> (batch seq) instances d_mlp", instances=2)
post_reshaped.shape

torch.Size([3, 2, 6400])

In [28]:
# sparse_autoencoder = next(iter(sparse_autoencoder_dictionary))[1]
sparse_autoencoder(post_reshaped)

RuntimeError: The size of tensor a (6400) must match the size of tensor b (1600) at non-singleton dimension 2

In [23]:
sparse_autoencoder_dictionary(post_reshaped)

TypeError: 'SparseAutoencoderDictionary' object is not callable

In [21]:
# use a fwd pass to compute ALL feature actvs for ALL this steering vec
# output_tuple = autoencoder.forward(post_reshaped)
acts = output_tuple[3]
acts.shape

AttributeError: 'SparseAutoencoderDictionary' object has no attribute 'forward'

In [ ]:
# Get the top k largest activations for feature neurons, not batch seq. use , dim=-1
feat_k = 5
top_acts_values, top_acts_indices = acts.topk(feat_k, dim=-1)

In [ ]:
top_acts_indices

tensor([[[  21, 3622, 3274, 4597, 1279],
         [ 144, 3016, 1272, 1681,  917]],

        [[ 939, 4847,  594, 5196, 1176],
         [ 623, 3874, 4619,  663, 1692]],

        [[   1,    0,    2,    4,    3],
         [   1,    0,    2,    4,    3]]], device='cuda:0')

In [ ]:
all_tokens = model.to_tokens(batch_input, prepend_bos=True)
all_tokens = all_tokens.to(device)
all_tokens.shape

torch.Size([100, 34])

In [ ]:
# get top samp_m tokens for all top feat_k feature neurons
samp_m = 5
for feature_idx in top_acts_indices[0][0]:
    feature_idx = feature_idx.item()
    ds_top_acts_indices, ds_top_acts_values = highest_activating_tokens(all_tokens, model, autoencoder, feature_idx,
                                                            autoencoder_B=False, k=samp_m, layer_name=layer_name)
    display_top_sequences(ds_top_acts_indices, ds_top_acts_values, all_tokens)

           Tokens which most activate feature 21            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sequence                                    ┃ Activation ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ at the time of this writing in early June)  │ -0.01      │
│  a dozen — "Americans for Real Good Coffee" │ -0.04      │
│ <|endoftext|>Anarchists in                  │ -0.11      │
│ <|endoftext|>For today��                    │ -0.11      │
│ <|endoftext|>A magazine supplement with     │ -0.11      │
└─────────────────────────────────────────────┴────────────┘

          Tokens which most activate feature 3622           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sequence                                    ┃ Activation ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ at the time of this writing in early June)  │ -0.04      │
│  a dozen — "Americans for Real Good Coffee" │ -0.07      │
│ <|endoftext|>Anarchists in                  │ -0.12      │
│ <|endoftext|>For today��                    │ -0.12      │
│ <|endoftext|>A magazine supplement with     │ -0.12      │
└─────────────────────────────────────────────┴────────────┘

                           Tokens which most activate feature 3274                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sequence                                                                      ┃ Activation ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│  a dozen — "Americans for Real Good Coffee"                                   │ -0.03      │
│ at the time of this writing in early June)                                    │ -0.04      │
│  the sauce in their carry-on luggage, e                                       │ -0.09      │
│  of the Johns Hopkins Blo<|endoftext|><|endoftext|><|endoftext|><|endoftext|> │ -0.09      │
│  the former J.L. Hudson��s                                                    │ -0.09      │
└───────────────────────────────────────────────────────────────────────────────┴────────────┘

            Tokens which most activate feature 4597            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sequence                                       ┃ Activation ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ at the time of this writing in early June)     │ -0.05      │
│  a dozen — "Americans for Real Good Coffee"    │ -0.06      │
│  the sauce in their carry-on luggage, e        │ -0.14      │
│ It's not clear how or why two men attacked     │ -0.15      │
│  The world's best make-up artists reveal their │ -0.18      │
└────────────────────────────────────────────────┴────────────┘

          Tokens which most activate feature 1279           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Sequence                                    ┃ Activation ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ at the time of this writing in early June)  │ -0.00      │
│  a dozen — "Americans for Real Good Coffee" │ -0.05      │
│ It's not clear how or why two men attacked  │ -0.10      │
│ <|endoftext|>For today��                    │ -0.12      │
│ <|endoftext|>A magazine supplement with     │ -0.12      │
└─────────────────────────────────────────────┴────────────┘